In [1]:
import pandas as pd
from xml.dom import minidom
from pyzabbix import ZabbixAPI
import json
zapi = ZabbixAPI('http://192.168.50.100:18080')
zapi.login('Admin','zabbix')

In [2]:
def create_element(text):
    return dom.createElement(text)

In [3]:
def create_text(text):
    return dom.createTextNode(text)

In [4]:
dom = minidom.Document()
root_node = create_element('zabbix_export')
dom.appendChild(root_node)

<DOM Element: zabbix_export at 0x7fd3ee9a40f0>

In [5]:
root_node.appendChild(create_element('version')).appendChild(create_text('5.0'))
root_node.appendChild(create_element('groups')).appendChild(create_element('group')).appendChild(create_element('name')).appendChild(create_text('IPMI'))
template = root_node.appendChild(create_element('templates')).appendChild(create_element('template'))
template.appendChild(create_element('template')).appendChild(create_text('Template Huawei of IPMI'))
template.appendChild(create_element('name')).appendChild(create_text('Template Huawei of IPMI'))
template.appendChild(create_element('groups')).appendChild(create_element('group')).appendChild(create_element('name')).appendChild(create_text('IPMI'))

<DOM Text node "'IPMI'">

In [6]:
item_list = zapi.item.get(
    hostids='10388',
    search = {
        "key_" : 'ipmi.get'
    },
    output="extend",
)

In [7]:
data = pd.DataFrame(json.loads(item_list[0]['lastvalue']))
data.to_json('./data.json')

In [8]:
pd.read_json('./data.json')

,id,name,sensor,reading,state,value,units,threshold
0,PS1 Temp Status,(r0.32.10.0).PS1 Temp Status,"{'type': 33, 'text': 'slot_connector'}","{'type': 111, 'text': 'sensor specific'}","{'state': 0, 'text': ''}",NaN,None,None
1,PS1 Fan Status,(r0.32.10.0).PS1 Fan Status,"{'type': 33, 'text': 'slot_connector'}","{'type': 111, 'text': 'sensor specific'}","{'state': 0, 'text': ''}",NaN,None,None
2,PS1 Status,(r0.32.10.0).PS1 Status,"{'type': 8, 'text': 'power_supply'}","{'type': 111, 'text': 'sensor specific'}","{'state': 1, 'text': 'presence detected'}",NaN,None,None
3,DISK7,(r0.32.4.7).DISK7,"{'type': 13, 'text': 'drive_slot'}","{'type': 111, 'text': 'sensor specific'}","{'state': 0, 'text': ''}",NaN,None,None
4,DISK6,(r0.32.4.6).DISK6,"{'type': 13, 'text': 'drive_slot'}","{'type': 111, 'text': 'sensor specific'}","{'state': 1, 'text': 'drive presence'}",NaN,None,None
...,...,...,...,...,...,...,...,...
134,CPU2 Core Rem,(r0.32.3.1).CPU2 Core Rem,"{'type': 1, 'text': 'temperature'}","{'type': 1, 'text': 'threshold'}","{'state': 0, 'text': ''}",46.0,C,"{'lower': {}, 'upper': {}}"
135,CPU1 Core Rem,(r0.32.3.0).CPU1 Core Rem,"{'type': 1, 'text': 'temperature'}","{'type': 1, 'text': 'threshold'}","{'state': 0, 'text': ''}",45.0,C,"{'lower': {}, 'upper': {}}"
136,PCH Temp,(r0.32.7.0).PCH Temp,"{'type': 1, 'text': 'temperature'}","{'type': 1, 'text': 'threshold'}","{'state': 0, 'text': ''}",57.0,C,"{'lower': {}, 'upper': {'non_crit': 86.0}}"
137,Outlet Temp,(r0.32.7.0).Outlet Temp,"{'type': 1, 'text': 'temperature'}","{'type': 1, 'text': 'threshold'}","{'state': 0, 'text': ''}",35.0,C,"{'lower': {}, 'upper': {'non_crit': 75.0}}"


In [9]:
applications = []
for i in data['sensor']:
    applications.append(i['text'])
applications = list(set(applications))
applications.append('IPMI_LLD')

In [10]:
template_applications = template.appendChild(create_element('applications'))
for application in applications:
    template_applications.appendChild(create_element('application')).appendChild(create_element('name')).appendChild(create_text(application))

In [11]:
applications.remove('IPMI_LLD')

In [12]:
template_item = template.appendChild(create_element('items')).appendChild(create_element('item'))
dict_item = {'name':'IPMI Discovery', 'type' : 'IPMI', 'key': 'ipmi.get', 'history' : '2h', 'value_type' : 'TEXT'}
for i in dict_item:
    template_item.appendChild(create_element(i)).appendChild(create_text(dict_item[i]))
template_item.appendChild(create_element('applications')).appendChild(create_element('application')).appendChild(create_element('name')).appendChild(create_text('IPMI_LLD'))

<DOM Text node "'IPMI_LLD'">

In [13]:
lld_macro_dict = {'{#READING_TYPE}':'$.reading.type', 
            '{#SENSOR_ID}':'$.id',
            '{#SENSOR_NAME}':'$.name',
            '{#SENSOR_TYPE_STR}':'$.sensor.text',
            '{#SENSOR_TYPE}':'$.sensor.type',
            '{#SENSOR_UNIT}':'$.units',
            '{#THRESH_HIGH_AVG}':'$.threshold.upper.crit',
            '{#THRESH_HIGH_CRIT}':'$.threshold.upper.non_recover',
            '{#THRESH_HIGH_WARN}':'$.threshold.upper.non_crit',
            '{#THRESH_LOW_AVG}':'$.threshold.lower.crit',
            '{#THRESH_LOW_CRIT}':'$.threshold.lower.non_recover',
            '{#THRESH_LOW_WARN}':'$.threshold.lower.non_crit'
           }

In [14]:
sensor_chinese = pd.read_json('./sensor.json',orient = 'index')
sensor_chinese = sensor_chinese.to_dict(orient = 'list')

In [15]:
template_discovery = template.appendChild(create_element('discovery_rules'))
for application in applications:
    discovery_rule = template_discovery.appendChild(create_element('discovery_rule'))
    discovery_rule.appendChild(create_element('name')).appendChild(create_text('IPMI Discovery: {}'.format(' '.join([i.capitalize() for i in application.split('_')]))))
    discovery_rule.appendChild(create_element('type')).appendChild(create_text('DEPENDENT'))
    discovery_rule.appendChild(create_element('key')).appendChild(create_text('ipmi.sensor.discovery.{}'.format(application)))
    discovery_rule.appendChild(create_element('delay')).appendChild(create_text('0'))
    discovery_rule.appendChild(create_element('lifetime')).appendChild(create_text('7d'))
    discovery_rule.appendChild(create_element('master_item')).appendChild(create_element('key')).appendChild(create_text('ipmi.get'))
    
    rule_filter = discovery_rule.appendChild(create_element('filter')).appendChild(create_element('conditions'))
    filter_condition = rule_filter.appendChild(create_element('condition'))
    filter_condition.appendChild(create_element('macro')).appendChild(create_text('{#SENSOR_TYPE}'))
    value_sensor = '^{}$'.format(str(list(set([i['type'] for i in data[data['sensor'].apply(lambda x: x['text'] == application)]['sensor']]))[0]))
    filter_condition.appendChild(create_element('value')).appendChild(create_text(value_sensor))
    filter_condition.appendChild(create_element('formulaid')).appendChild(create_text('B'))

    filter_condition = rule_filter.appendChild(create_element('condition'))
    filter_condition.appendChild(create_element('macro')).appendChild(create_text('{#READING_TYPE}'))
    reading = list(set([i['type'] for i in data[data['sensor'].apply(lambda x: x['text'] == application)]['reading']]))
    if len(reading) == 1:
        value_sensor = '^{}$'.format(reading[0])
    else:
        value_sensor = '^({})$'.format('|'.join([str(i) for i in reading]))
    filter_condition.appendChild(create_element('value')).appendChild(create_text(value_sensor))
    filter_condition.appendChild(create_element('formulaid')).appendChild(create_text('A'))
    item_prototypes = discovery_rule.appendChild(create_element('item_prototypes'))
    item_prototype = item_prototypes.appendChild(create_element('item_prototype'))
    sensor = data[data['reading'].apply(lambda x : x['text'] == 'threshold')]['sensor']
    
    application_chinese = [sensor_chinese[i][0] for i in sensor_chinese if application == i]
    if application_chinese:
        state_description = application_chinese[0]+'传感器ID: {#SENSOR_ID}'+application_chinese[0]+'传感器名字: {#SENSOR_NAME}'+application_chinese[0]+'传感器状态: {#STATE_TEXT}'
        threshold_description = application_chinese[0]+'传感器ID: {#SENSOR_ID}\n' + application_chinese[0] +'传感器名字: {#SENSOR_NAME}\n' + application_chinese[0]+'传感器阈值\n'+'严重告警: {#THRESH_HIGH_WARN} , {#THRESH_HIGH_AVG} , {#THRESH_HIGH_CRIT} \n轻微告警: {#THRESH_LOW_WARN} , {#THRESH_LOW_AVG} , {#THRESH_LOW_CRIT}'
    else:
        application_chinese = [' ']
        state_description = application_chinese[0]+'传感器ID: {#SENSOR_ID}'+application_chinese[0]+'传感器名字: {#SENSOR_NAME}'+application_chinese[0]+'传感器状态: {#STATE_TEXT}'
        threshold_description = application_chinese[0]+'传感器ID: {#SENSOR_ID}\n' + application_chinese[0] +'传感器名字: {#SENSOR_NAME}\n' + application_chinese[0]+'传感器阈值\n'+'严重告警: {#THRESH_HIGH_WARN} , {#THRESH_HIGH_AVG} , {#THRESH_HIGH_CRIT} \n轻微告警: {#THRESH_LOW_WARN} , {#THRESH_LOW_AVG} , {#THRESH_LOW_CRIT}'
        
    preprocessing = item_prototype.appendChild(create_element('preprocessing')).appendChild(create_element('step'))
    preprocessing.appendChild(create_element('type')).appendChild(create_text('JSONPATH'))
    item_prototype.appendChild(create_element('name')).appendChild(create_text(application_chinese[0] + '传感器: {#SENSOR_ID}的当前状态值'))
    item_prototype.appendChild(create_element('description')).appendChild(create_text(state_description))
    preprocessing.appendChild(create_element('params')).appendChild(create_text("$.[?(@.id=='{#SENSOR_ID}')].state.state.first()"))        
    item_prototype.appendChild(create_element('type')).appendChild(create_text('DEPENDENT'))
    item_prototype.appendChild(create_element('key')).appendChild(create_text('ipmi.sensor.'+ application +'[{#SENSOR_ID}]'))
    item_prototype.appendChild(create_element('delay')).appendChild(create_text('0'))
    item_prototype.appendChild(create_element('history')).appendChild(create_text('7d'))
    item_prototype.appendChild(create_element('applications')).appendChild(create_element('application')).appendChild(create_element('name')).appendChild(create_text(application))
    item_prototype.appendChild(create_element('master_item')).appendChild(create_element('key')).appendChild(create_text('ipmi.get'))
    
    if application in [i['text'] for i in sensor]:
        item_prototype = item_prototypes.appendChild(create_element('item_prototype'))
        item_prototype.appendChild(create_element('name')).appendChild(create_text(application_chinese[0] + '传感器: {#SENSOR_ID}的当前值'))
        item_prototype.appendChild(create_element('description')).appendChild(create_text(threshold_description))
        if application == 'temperature':
            item_prototype.appendChild(create_element('units')).appendChild(create_text('℃')) 
        else:
            units = list(set(data[data['sensor'].apply(lambda x :x['text'] == application)]['units']))[0]
            item_prototype.appendChild(create_element('units')).appendChild(create_text(units))  
        preprocessing = item_prototype.appendChild(create_element('preprocessing')).appendChild(create_element('step'))
        preprocessing.appendChild(create_element('type')).appendChild(create_text('JSONPATH'))
        preprocessing.appendChild(create_element('params')).appendChild(create_text("$.[?(@.id=='{#SENSOR_ID}')].value.first()"))
        item_prototype.appendChild(create_element('type')).appendChild(create_text('DEPENDENT'))
        item_prototype.appendChild(create_element('key')).appendChild(create_text('ipmi.sensor.'+ application +'[{#SENSOR_ID}]'))
        item_prototype.appendChild(create_element('delay')).appendChild(create_text('0'))
        item_prototype.appendChild(create_element('history')).appendChild(create_text('7d'))
        item_prototype.appendChild(create_element('applications')).appendChild(create_element('application')).appendChild(create_element('name')).appendChild(create_text(application))
        item_prototype.appendChild(create_element('master_item')).appendChild(create_element('key')).appendChild(create_text('ipmi.get'))

    
    lld_macros = discovery_rule.appendChild(create_element('lld_macro_paths'))
    for macro in lld_macro_dict:
        lld_macro = lld_macros.appendChild(create_element('lld_macro_path'))
        lld_macro.appendChild(create_element('lld_macro')).appendChild(create_text(macro))
        lld_macro.appendChild(create_element('path')).appendChild(create_text(lld_macro_dict[macro]))

In [16]:
try:
    with open('./huawei_ipmi.xml','w',encoding='UTF-8') as fh:
        dom.writexml(fh,indent='',addindent='\t',newl='\n',encoding='UTF-8')
        print('写入xml OK!')
except Exception as err:
    print('错误信息：{0}'.format(err))

写入xml OK!
